# Selenium Usage

反爬設置：若需避免反爬，可以加上自訂的 User-Agent 或模擬人類行為（如隨機點擊、等待）

1. 設置自訂的 User-Agent

    許多網站會檢測 User-Agent，如果發現是無頭瀏覽器，可能會拒絕訪問。可以通過 chrome_options 設置自訂的 User-Agent。

        from selenium.webdriver.chrome.options import Options

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.198 Safari/537.36")


2. 禁用 Selenium 的特徵標記
    某些網站會檢測瀏覽器中的特徵標記（如 navigator.webdriver），以此識別 Selenium。可以通過修改 JavaScript 屬性來隱藏這些標記。

        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            """
        })


3. 隨機延遲操作 
    爬取時添加隨機延遲來模仿人類操作，而不是直接快速執行每個動作。
        import time
        import random
        def random_delay():
            time.sleep(random.uniform(2, 5))  # 隨機延遲 2 到 5 秒
        使用時：
        random_delay()
        driver.get("https://example.com")
        random_delay()


4. 設置瀏覽器窗口大小
    某些網站會檢測瀏覽器窗口大小，無頭模式默認使用最小窗口，可以手動設置為普通大小。
        chrome_options.add_argument("window-size=1920,1080")

5. 禁用無頭模式標記
    某些網站能識別到無頭瀏覽器，可以通過禁用一些無頭模式特徵來隱藏。
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")

6. 模擬鼠標和鍵盤操作 通過 Selenium 的 ActionChains 模組模擬滑鼠移動、點擊和鍵盤輸入。

        from selenium.webdriver.common.action_chains import ActionChains
        from selenium.webdriver.common.keys import Keys

        action = ActionChains(driver)
        element = driver.find_element("name", "q")  # 找到 Google 搜索框
        action.move_to_element(element).click().send_keys("Selenium").send_keys(Keys.RETURN).perform()


7. 使用代理 IP
    使用代理 IP 可以減少目標網站對於同一 IP 的訪問限制。
        
        chrome_options.add_argument("--proxy-server=http://your-proxy-server:port")
        
        若需要動態代理，建議使用服務如 ProxyMesh 或 Bright Data。

8. 多用戶代理切換
    通過腳本自動切換不同的代理 IP，減少單一 IP 的檢測風險。

# Install packages

In [5]:
# 安裝所需套件
# !pip install selenium webdriver-manager
# !pip install fake-useragent -U


In [6]:

# 安裝 Google Chrome if your are using Colab
# !apt-get update
# !apt-get install -y wget
# !wget -q -O google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !dpkg -i google-chrome-stable_current_amd64.deb
# !apt-get -f install -y


In [7]:

# 實作程式碼
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import random
# 
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as Soup
import pandas as pd


In [8]:
# 隨機延遲 (random_delay)
# 模擬人類操作的速度，避免連續的快速請求引起網站注意。

# 自訂 User-Agent
# 模仿真實的瀏覽器標識，避免被識別為自動化工具。

# 隱藏 Selenium 的特徵
# 使用 JavaScript 修改 navigator.webdriver，防止網站檢測到 Selenium 的執行環境。

# 設置窗口大小
# 無頭模式下默認窗口大小可能會暴露機器特徵，設置常見的分辨率如 1920x1080。

# 模擬輸入與提交
# 使用 Selenium 的 send_keys 和 submit 進行互動操作，模擬用戶行為。

#-------------------
# 隨機延遲函數
def random_delay():
    time.sleep(random.uniform(5, 15))  # 隨機延遲 2 到 5 秒

# 設定 Chrome Options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 無頭模式 不顯示瀏覽器界面
chrome_options.add_argument("--no-sandbox") # 避免沙盒
chrome_options.add_argument("--incognito") # 無痕模式
chrome_options.add_argument("--disable-dev-shm-usage") # 避免記憶體不足
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 隱藏無頭模式特徵
chrome_options.add_argument("window-size=1920,1080")  # 模擬普通瀏覽器視窗大小
# 禁止印出log、隱藏受自動化軟體控制之資訊
# 禁用Chrome的自動化擴充功能
# 可以避免某些網站檢測到自動化測試而阻擋訪問
# 提高爬蟲的穩定性和真實性
# 這些設定的主要目的是:

# 讓自動化程式更像真實用戶在使用瀏覽器
# 避免被網站檢測出是自動化工具
# 減少不必要的日誌輸出
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging', 'enable-automation'])
chrome_options.add_experimental_option('useAutomationExtension', False)

# 許多網站會檢測 User-Agent，如果發現是無頭瀏覽器，可能會拒絕訪問。可以通過 chrome_options 設置自訂的 User-Agent。
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.198 Safari/537.36"
)  # 設定自訂 User-Agent

# 自動下載與配置 ChromeDriver 它會根據你的 Chrome 瀏覽器版本自動匹配合適的 ChromeDriver，免除手動下載與管理
#service = Service(ChromeDriverManager().install())
# 啟動 Selenium WebDriver
#driver = webdriver.Chrome(service=service, options=chrome_options)

# 啟動 Selenium WebDriver
try:
    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(
        service=service,
        options=chrome_options
    )
except Exception as e:
    print(f"Error initializing Chrome browser: {e}")
    raise # Keep this to ensure program stops on critical error


# 禁用 Selenium 的特徵標記
# 隱藏 Selenium 特徵
browser.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """
        Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
        """
    },
)


{'identifier': '2'}

In [ ]:
full_url = 'https://www.cna.com.tw/list/aipl.aspx' #政治
# full_url = 'https://www.cna.com.tw/list/aopl.aspx' #國際
#full_url = 'https://www.cna.com.tw/list/ahel.aspx' #生活

#target_date_str = '2025-03-31'
#target_date = datetime.strptime(target_date_str, '%Y-%m-%d')

# target_date = datetime.now() # 取得當前日期
target_date = datetime.now() - timedelta(days=1) # 取得昨天日期
# target_date = datetime.now() - timedelta(days=2) # 取得前天日期


try:
    # 開啟網頁
    browser.get(full_url)
    # 隨機延遲
    random_delay()
    
    # 按隱私權按鈕
    '''
    <a class="closeBtn close-privacy" href="javascript:" aria-label="同意並關閉隱私權"><i class="icon-cross"></i></a>

    '''
    # try:
    #     #driver.get('https://www.cna.com.tw/')
    #     #element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a#closeBtn close-privacy"))).click()
    #     element = WebDriverWait(browser, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "closeBtn.close-privacy")))
    #     element[0].click()
    #     print('按隱私權按鈕OK')
    # except Exception as ex:
    #     print('按隱私權按鈕失敗!',ex)
        
    page = 1
    while True:        
        
        # 滾輪到最下面
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print('滾輪到最下面OK')
        
        # Google需搭配新版的Selenium--新版modified on March 8, 2023
        try:
            # 等待元素可見
            element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.ID, "SiteContent_uiViewMoreBtn_Style3")))
            browser.find_element(By.ID,"SiteContent_uiViewMoreBtn_Style3").click()
            print('按更多新聞OK')
            print(f'翻頁{page}次')
            page += 1
        except Exception as ex:
            # 最後一頁沒有更多新聞了，沒有按鈕可按，會拋錯!
            # print('按更多新聞失敗!',ex)
            print('最後一頁沒有更多新聞了，沒有按鈕可按，拋錯!')
            break

        # Here we can control the number of pages to scroll.
        # 可以只翻3次pages
        # if (page >= 3): 
        #     break 
        random_delay()  # 隨機延遲
        
    print('開始抓新聞列表')    
    # 取得頁面HTML
    html = browser.page_source
    # 取得新聞列表內容
    # (1) Soup版本
    # item_list_soup = BeautifulSoup(html, 'lxml')
    # items = item_list_soup.select('ul.mainList li a')
    # (2) Selenium版本
    items = browser.find_elements(By.CSS_SELECTOR, 'ul.mainList li a')

    # 取得新聞列表內容，逐項處理
    serial_no=1
    for item in items:
        
        # 抓日期
        try:
            # (1) Soup版本
            # news_time_str = item.find('div', {'class':"date"}).text #抓日期時間字串有包含(時:分)
            # (2) Selenium版本
            # In Selenium, you'd use find_element instead of find
            news_time_str = item.find_element(By.CSS_SELECTOR, 'div.date').text
        
            # 排除穿插的廣告 其日期有錯
            news_dtime = datetime.strptime(news_time_str, '%Y/%m/%d %H:%M') #轉成datetime格式
        except:
            print('排除這則新聞:',item)
            continue

        # article date
        news_date_str = news_dtime.strftime("%Y-%m-%d")

        # 新聞時間過濾，你若要爬某一天的新聞，必須過濾。若要爬取全部，則將以下兩行指令取消即可。
        # 排除不是目標日期的新聞 兩者日期相比對
        
        if target_date.date() != news_dtime.date():
            # print(f"新聞日期不符合，跳開這一則{item['href']}")
            continue

        # date
        print(news_date_str)

        # title
        # (1) Soup版本
        # title = item.find('h2').find('span').text
        # (2) Selenium版本
        title = item.find_element(By.CSS_SELECTOR, 'h2 span').text
        print(title)
        
finally:
    # 關閉瀏覽器
    browser.quit()
    # pass
    print('關閉瀏覽器OK')


按隱私權按鈕OK
滾輪到最下面OK
按更多新聞OK
翻頁1次
滾輪到最下面OK
按更多新聞OK
翻頁2次
滾輪到最下面OK
按更多新聞OK
翻頁3次
滾輪到最下面OK
按更多新聞OK
翻頁4次
滾輪到最下面OK
最後一頁沒有更多新聞了，沒有按鈕可按，拋錯!
開始抓新聞列表
2025-03-31
退輔會建置「軍民專長轉銜對照表」拚2027年啟用
2025-03-31
核管法表決戰 綠質疑不敢面對點名、藍批就是在鬧事
2025-03-31
0403地震將滿週年 國土署：核撥受災戶租補逾1億元
2025-03-31
孟加拉領導人喊一中 外交部：謬論無法改變兩岸現狀
2025-03-31
立院國發基金調查委員會 國發會說明投資機制5大改革
2025-03-31
權利回復基金會通過59件申請案 賠償逾1.7億元
2025-03-31
張亞光調升駐葡萄牙代表 胡琪斌任科威特代表
2025-03-31
國史館公開第5批兩蔣日記 記載二戰開始與結束
2025-03-31
綠促核管法停止審查未果 藍提延長核電廠運轉送協商
2025-03-31
軍公教加薪待入帳 政院盼約用人員一併調薪擬提追加預算
2025-03-31
清明前夕祭史明 蔡英文：攜手團結做勇敢的台灣人
2025-03-31
鍾文智棄保潛逃 黃國昌：世紀司法大醜聞應嚴肅究責
2025-03-31
罷免藍委行動車啟動掃街 曹興誠盼走遍全台角落
2025-03-31
劉世芳指預算凍刪將影響租金補貼 國民黨：公然造謠
2025-03-31
空軍戰術總驗收順利落幕 戰技競賽下半年登場
2025-03-31
立院初審紀念日及節日實施條例 關鍵條文全保留協商
2025-03-31
台積電高雄2奈米擴產 卓榮泰：布局全球根留台灣不變
2025-03-31
台灣相撲女將赴日征戰 立院女子運動促進會力挺
2025-03-31
國史館：鄭南榕獄中日記新書發表 4/7辦座談會
2025-03-31
賴總統印尼文發文 祝賀穆斯林開齋節快樂
2025-03-31
內政部核發溝子埧產業園區開發許可 估年產值99億元
2025-03-31
陸配涉武統言論限期離境 吳思瑤：勿挑戰公權力
2025-03-31
藍控卓榮泰戒嚴國會 綠問國民黨為何幫中國滲透台灣
2025-03-31
川普將開徵對等關稅 經貿辦：已研擬多面向因應方案
2025-03-31
賴總